# Malta Covid Data Analysys

## Import Libraries

In [267]:
import pandas as pd
import numpy as np
import calendar
import plotly.express as px

## Load Datasets

In [298]:
cases = pd.read_csv('../data/raw/COVID-19 Malta - Aggregate Data Set.csv')
tests = pd.read_csv('../data/raw/COVID-19 Malta - COVID Tests.csv')
vaccines = pd.read_csv('../data/raw/COVID-19 Malta - Vaccination Data.csv')
hospitalized = pd.read_csv('../data/raw/COVID-19 EU - Daily Hospitalized.csv')

## Group Covid Cases Dataset

In [234]:
cases_df = pd.DataFrame()
cases_df['Date'] = pd.to_datetime(cases['Date'], format='%d/%m/%Y')
cases_df = cases.groupby(cases_df.Date.dt.to_period('M')).agg({
    'New Cases': 'sum',
    'Total Cases': 'last',
    'Recovered': 'last',
    'Deaths': 'last',
    'Active Cases': 'last'
})

cases_df.insert(0, 'Date', cases_df.index.strftime('%m/%Y'))
cases_df.insert(1, 'Year', cases_df.index.year)
cases_df.insert(2, 'Month', cases_df.index.month)

cases_df = cases_df.reset_index(drop=True)

cases_df['Month'] = cases_df['Month'].apply(lambda x: calendar.month_abbr[x])

def get_diff(row, column):
    if row.name == 0:
        return row[column]
    return row[column] - cases_df.iloc[row.name - 1][column]

cases_df.insert(6, 'New Recovered', cases_df.apply(get_diff, axis=1, column='Recovered'))
cases_df.insert(8, 'New Deaths', cases_df.apply(get_diff, axis=1, column='Deaths'))

cases_df.rename(columns={
    'Recovered': 'Total Recovered',
    'Deaths': 'Total Deaths'
}, inplace=True)

cases_df

,Date,Year,Month,New Cases,Total Cases,Total Recovered,New Recovered,Total Deaths,New Deaths,Active Cases
0,03/2020,2020,Mar,169,169,2,2,0,0,167
1,04/2020,2020,Apr,296,465,351,349,4,4,110
2,05/2020,2020,May,153,618,534,183,9,5,75
3,06/2020,2020,Jun,52,670,640,106,9,0,21
4,07/2020,2020,Jul,154,824,665,25,9,0,150
5,08/2020,2020,Aug,1164,1883,1400,735,12,3,471
6,09/2020,2020,Sep,1175,3058,2562,1162,34,22,462
7,10/2020,2020,Oct,2984,6042,4155,1593,62,28,1825
8,11/2020,2020,Nov,3831,9873,7665,3510,137,75,2071
9,12/2020,2020,Dec,2901,12774,11121,3456,219,82,1434


## Group Tests Dataset

In [256]:
tests_df = pd.DataFrame({
    'Date': tests['Publication date'],
    'Tests in prev day': tests['NAA and rapid antigen tests in previous day'],
    'Total Tests': tests['Total NAA and rapid antigen tests']
})
tests_df = tests_df.apply(
    lambda row: 
        row if row.name == len(tests_df.index) - 1 
            else [
                row['Date'], 
                tests_df.iloc[row.name + 1]['Tests in prev day'], 
                tests_df.iloc[row.name + 1]['Total Tests']], 
    axis=1, 
    result_type='broadcast'
)
tests_df = tests_df.rename(columns={
    'Tests in prev day': 'New Tests'
})
tests_df['Date'] = pd.to_datetime(tests_df['Date'], format='%d/%m/%Y')
tests_df = tests_df.groupby(tests_df.Date.dt.to_period('M')).agg({
    'New Tests': 'sum',
    'Total Tests': 'last'
})
tests_df.insert(0, 'Date', tests_df.index.strftime('%m/%Y'))
tests_df = tests_df.reset_index(drop=True)
tests_df

,Date,New Tests,Total Tests
0,07/2020,14927,128747
1,08/2020,63762,192509
2,09/2020,64048,256557
3,10/2020,83176,339733
4,11/2020,91049,430782
5,12/2020,86754,517536
6,01/2021,98081,615617
7,02/2021,86493,702110
8,03/2021,108992,811102
9,04/2021,57961,869063


## Group Vaccines Dataset

In [297]:
vaccines_df = vaccines.rename(columns={
    'Date of Vaccination': 'Date',
    'Primary Vaccination': 'Total Primary Vaccination',
    'Received one dose': 'Total Received one dose'
})
vaccines_df['Date'] = pd.to_datetime(vaccines_df['Date'], format='%d/%m/%Y')
vaccines_df = vaccines_df.groupby(vaccines_df.Date.dt.to_period('M')).agg({
    'Total Vaccination Doses': 'last',
    'Total Primary Vaccination': 'last',
    'Total Received one dose': 'last',
    'Total Booster doses': 'last',
    'Total 2nd Booster doses': 'last',
    'Omicron booster doses': 'sum',
    'Total Omicron booster doses': 'last'
})
vaccines_df = vaccines_df.replace({np.nan: 0})
vaccines_df = vaccines_df.astype(np.int64)

vaccines_df.insert(0, 'Date', vaccines_df.index.strftime('%m/%Y'))
vaccines_df = vaccines_df.reset_index(drop=True)

def get_diff(row, column):
    if row.name == 0:
        return row[column]
    return row[column] - vaccines_df.iloc[row.name - 1][column]

vaccines_df.insert(2, 'New Vaccination Doses', vaccines_df.apply(get_diff, axis=1, column='Total Vaccination Doses'))
vaccines_df.insert(4, 'New Primary Vaccination', vaccines_df.apply(get_diff, axis=1, column='Total Primary Vaccination'))
vaccines_df.insert(6, 'New Received one dose', vaccines_df.apply(get_diff, axis=1, column='Total Received one dose'))
vaccines_df.insert(8, 'New Booster doses', vaccines_df.apply(get_diff, axis=1, column='Total Booster doses'))
vaccines_df.insert(10, 'New 2nd Booster doses', vaccines_df.apply(get_diff, axis=1, column='Total 2nd Booster doses'))

vaccines_df.insert(13, 'New Omicron booster doses', vaccines_df['Omicron booster doses'])
vaccines_df = vaccines_df.drop(columns=['Omicron booster doses'])

vaccines_df

,Date,Total Vaccination Doses,New Vaccination Doses,Total Primary Vaccination,New Primary Vaccination,Total Received one dose,New Received one dose,Total Booster doses,New Booster doses,Total 2nd Booster doses,New 2nd Booster doses,Total Omicron booster doses,New Omicron booster doses
0,01/2021,27759,27759,3948,3948,23811,23811,0,0,0,0,0,0
1,02/2021,79624,51865,28048,24100,51576,27765,0,0,0,0,0,0
2,03/2021,197383,117759,54578,26530,142805,91229,0,0,0,0,0,0
3,04/2021,335848,138465,107038,52460,228810,86005,0,0,0,0,0,0
4,05/2021,519994,184146,210486,103448,318743,89933,0,0,0,0,0,0
5,06/2021,670759,150765,325891,115405,359169,40426,0,0,0,0,0,0
6,07/2021,759818,89059,382255,56364,399690,40521,0,0,0,0,0,0
7,08/2021,797017,37199,412767,30512,413985,14295,0,0,0,0,0,0
8,09/2021,824327,27310,421713,8946,421602,7617,12071,12071,0,0,0,0
9,10/2021,879909,55582,428333,6620,430283,8681,52915,40844,0,0,0,0


## Hospital Dataset

In [351]:
malta_hospitalized = hospitalized.loc[hospitalized['country'] == 'Malta'].drop(columns=['year_week', 'source', 'url', 'country'])
malta_hospitalized = malta_hospitalized.rename(columns={'date': 'Date'})
malta_hospitalized['value'] = malta_hospitalized['value'].astype(np.int64)
malta_hospitalized['Date'] = pd.to_datetime(malta_hospitalized['Date'], format='%Y-%m-%d')

### Group Hosptialized Data

In [360]:
hospitalized_df = malta_hospitalized.loc[malta_hospitalized['indicator'] == 'Daily hospital occupancy'].drop(columns=['indicator'])
hospitalized_df = hospitalized_df.rename(columns={'value': 'Hospitalized'})
hospitalized_df = hospitalized_df.reset_index(drop=True)

def calc_new_hospitalized(row):
    if row.name == 0 or hospitalized_df.iloc[row.name - 1]['Hospitalized'] >= row['Hospitalized']:
        return 0
    return row['Hospitalized'] - hospitalized_df.iloc[row.name - 1]['Hospitalized']

hospitalized_df['New Hospitalized'] = hospitalized_df.apply(calc_new_hospitalized, axis=1)
hospitalized_df['Total Hospitalized'] = hospitalized_df['New Hospitalized'].cumsum()

hospitalized_df = hospitalized_df.groupby(hospitalized_df.Date.dt.to_period('M')).agg({
    'Hospitalized': 'last',
    'New Hospitalized': 'sum',
    'Total Hospitalized': 'last'
})
hospitalized_df.insert(0, 'Date', hospitalized_df.index.strftime('%m/%Y'))
hospitalized_df = hospitalized_df.reset_index(drop=True)
hospitalized_df

,Date,Hospitalized,New Hospitalized,Total Hospitalized
0,03/2020,9,16,16
1,04/2020,4,12,28
2,05/2020,3,9,37
3,06/2020,0,10,47
4,07/2020,1,1,48
5,08/2020,7,33,81
6,09/2020,22,35,116
7,10/2020,53,63,179
8,11/2020,80,85,264
9,12/2020,66,56,320


### Group ICU Data

In [362]:
icu_df = malta_hospitalized.loc[malta_hospitalized['indicator'] == 'Daily ICU occupancy'].drop(columns=['indicator'])
icu_df = icu_df.rename(columns={'value': 'ICU'})
icu_df = icu_df.reset_index(drop=True)

def calc_new_icu(row):
    if row.name == 0 or icu_df.iloc[row.name - 1]['ICU'] >= row['ICU']:
        return 0
    return row['ICU'] - icu_df.iloc[row.name - 1]['ICU']

icu_df['New ICU'] = icu_df.apply(calc_new_icu, axis=1)
icu_df['Total ICU'] = icu_df['New ICU'].cumsum()

icu_df = icu_df.groupby(icu_df.Date.dt.to_period('M')).agg({
    'ICU': 'last',
    'New ICU': 'sum',
    'Total ICU': 'last'
})
icu_df.insert(0, 'Date', icu_df.index.strftime('%m/%Y'))
icu_df = icu_df.reset_index(drop=True)
icu_df

,Date,ICU,New ICU,Total ICU
0,03/2020,4,4,4
1,04/2020,1,2,6
2,05/2020,2,3,9
3,06/2020,0,0,9
4,07/2020,0,0,9
5,08/2020,0,0,9
6,09/2020,5,5,14
7,10/2020,7,3,17
8,11/2020,17,20,37
9,12/2020,10,3,40


## Total Cases, Recovered & Deaths

In [202]:
fig = px.line(
    cases_df, 
    x='Date', 
    y=['Total Cases', 'Total Recovered', 'Total Deaths'], 
    title='Total Cases, Recovered & Deaths'
)
fig.update_layout(
    legend_title='Legend',
    yaxis_title='Cases'
)
fig.show()

### Total Deaths

In [204]:
fig = px.line(cases_df, x='Date', y='Total Deaths', title='Total Deaths')
fig.update_layout(
    yaxis_title='Deaths'
)
fig.show()